## Lab 1: Creating a Multi-Agent Customer Support System

### Overview

In this enhanced version of Lab 1, we'll build a **multi-agent customer support system** that demonstrates agent orchestration and specialization. Instead of a single agent handling all tasks, we'll create:

1. **Orchestrator Agent** - Routes customer requests to appropriate specialized agents
2. **Customer Support Agent** - Handles product info, returns, and web search
3. **Knowledge Base Agent** - Specialized agent for technical support using Bedrock Knowledge Base

This architecture mirrors real-world enterprise systems where different agents specialize in specific domains while an orchestrator coordinates the overall customer experience.


### Agent Responsibilities

#### 🎯 **Orchestrator Agent**
- **Purpose**: Intelligent request routing and coordination
- **Tools**: Agent communication and routing tools
- **Decisions**: Analyze customer queries and route to appropriate specialist

#### 🛠️ **Customer Support Agent** 
- **Purpose**: General customer support tasks
- **Tools**: `get_product_info()`, `get_return_policy()`, `web_search()`
- **Specialization**: Product information, policies, general troubleshooting

#### 📚 **Knowledge Base Agent**
- **Purpose**: Technical support and documentation
- **Tools**: `get_technical_support()` (Bedrock Knowledge Base)
- **Specialization**: Deep technical knowledge, complex troubleshooting

### Prerequisites

* **AWS Account** with appropriate permissions
* **Python 3.10+** installed locally
* **AWS CLI configured** with credentials
* **Amazon Nova Pro** enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* **Strands Agents** and other libraries installed in the next cells

### Step 1: Install Dependencies and Import Libraries
Before we start, let's install the pre-requisites for this lab. Please also ensure that you have run the steps in the terminal as part of the [Amazon SageMaker AI Prerequisites](https://catalog.us-east-1.prod.workshops.aws/workshops/2c21bfce-ff39-47af-bc4f-d6c21b03c093/en-US/03-sagemaker-studio).

In [ ]:
# Install dependencies for multi-agent system
print("🔧 Installing dependencies for Re:Invent Multi-Agent Tutorial...")
%pip install -r requirements.txt --upgrade-strategy only-if-needed -q
print("✅ Dependencies installed successfully!")
print("🚀 Ready to start Lab 1: Multi-Agent System Foundation")

In [ ]:
# Import libraries for multi-agent system
print("📚 Importing multi-agent system libraries...")

import json
import uuid
from typing import Dict, List, Any
from enum import Enum

from strands import Agent, tool
from strands.models import BedrockModel
from boto3.session import Session

# Set up basic configuration
boto_session = Session()
REGION = boto_session.region_name
MODEL_ID = "us.amazon.nova-pro-v1:0"

print("✅ Libraries imported successfully!")
print(f"🌍 Working in region: {REGION}")
print(f"🤖 Multi-agent system ready for Lab 1")

In [ ]:
# Validate Amazon Nova Pro model access
print("🧪 Validating Amazon Nova Pro model access...")
try:
    test_model = BedrockModel(model_id=MODEL_ID, temperature=0.1, region_name=REGION)
    print("✅ Amazon Nova Pro access confirmed")
    print(f"📋 Using model: {MODEL_ID}")
except Exception as e:
    print(f"❌ Amazon Nova Pro model access failed: {e}")
   

In [ ]:
# Define agent tools directly in the notebook
print("🛠️ Creating agent-specific tools...")

@tool(
    name="get_product_info",
    description="Get detailed technical specifications and information for electronics products"
)
def get_product_info(product_type: str) -> str:
    """
    Get detailed technical specifications and information for electronics products.

    Args:
        product_type: Electronics product type (e.g., 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        Formatted product information including warranty, features, and policies
    """
    # Mock product catalog - in real implementation, this would query a product database
    products = {
        "laptops": {
            "warranty": "1-year manufacturer warranty + optional extended coverage",
            "specs": "Intel/AMD processors, 8-32GB RAM, SSD storage, various display sizes",
            "features": "Backlit keyboards, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux support varies by model",
            "support": "Technical support and driver updates included"
        },
        "smartphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "5G/4G connectivity, 128GB-1TB storage, multiple camera systems",
            "features": "Wireless charging, water resistance, biometric security",
            "compatibility": "iOS/Android, carrier unlocked options available",
            "support": "Software updates and technical support included"
        },
        "headphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "Wired/wireless options, noise cancellation, 20Hz-20kHz frequency",
            "features": "Active noise cancellation, touch controls, voice assistant",
            "compatibility": "Bluetooth 5.0+, 3.5mm jack, USB-C charging",
            "support": "Firmware updates via companion app"
        },
        "monitors": {
            "warranty": "3-year manufacturer warranty",
            "specs": "4K/1440p/1080p resolutions, IPS/OLED panels, various sizes",
            "features": "HDR support, high refresh rates, adjustable stands",
            "compatibility": "HDMI, DisplayPort, USB-C inputs",
            "support": "Color calibration and technical support"
        }
    }
    product = products.get(product_type.lower())
    if not product:
        return f"Technical specifications for {product_type} not available. Please contact our technical support team for detailed product information and compatibility requirements."

    return f"Technical Information - {product_type.title()}:\n\n" \
           f"• Warranty: {product['warranty']}\n" \
           f"• Specifications: {product['specs']}\n" \
           f"• Key Features: {product['features']}\n" \
           f"• Compatibility: {product['compatibility']}\n" \
           f"• Support: {product['support']}"

@tool(
    name="get_return_policy",
    description="Get return policy information for a specific product category"
)
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information for a specific product category.

    Args:
        product_category: Electronics category (e.g., 'smartphones', 'laptops', 'accessories')

    Returns:
        Formatted return policy details including timeframes and conditions
    """
    # Mock return policy database - in real implementation, this would query policy database
    return_policies = {
        "smartphones": {
            "window": "30 days",
            "condition": "Original packaging, no physical damage, factory reset required",
            "process": "Online RMA portal or technical support",
            "refund_time": "5-7 business days after inspection",
            "shipping": "Free return shipping, prepaid label provided",
            "warranty": "1-year manufacturer warranty included"
        },
         "laptops": {
            "window": "30 days", 
            "condition": "Original packaging, all accessories, no software modifications",
            "process": "Technical support verification required before return",
            "refund_time": "7-10 business days after inspection",
            "shipping": "Free return shipping with original packaging",
            "warranty": "1-year manufacturer warranty, extended options available"
        },
        "accessories": {
            "window": "30 days",
            "condition": "Unopened packaging preferred, all components included",
            "process": "Online return portal",
            "refund_time": "3-5 business days after receipt",
            "shipping": "Customer pays return shipping under $50",
            "warranty": "90-day manufacturer warranty"
        }
    }

    # Default policy for unlisted categories
    default_policy = {
        "window": "30 days",
        "condition": "Original condition with all included components",
        "process": "Contact technical support",
        "refund_time": "5-7 business days after inspection", 
        "shipping": "Return shipping policies vary",
        "warranty": "Standard manufacturer warranty applies"
    }

    policy = return_policies.get(product_category.lower(), default_policy)
    return f"Return Policy - {product_category.title()}:\n\n" \
           f"• Return window: {policy['window']} from delivery\n" \
           f"• Condition: {policy['condition']}\n" \
           f"• Process: {policy['process']}\n" \
           f"• Refund timeline: {policy['refund_time']}\n" \
           f"• Shipping: {policy['shipping']}\n" \
           f"• Warranty: {policy['warranty']}"

@tool(
    name="web_search",
    description="Search the web for updated information"
)
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

@tool(
    name="get_technical_support",
    description="Get technical support information from knowledge base"
)
def get_technical_support(issue: str) -> str:
    """Get technical support solutions"""
    solutions = {
        "overheating": "Check ventilation, clean fans, monitor CPU usage, consider thermal paste replacement",
        "battery": "Calibrate battery, check power settings, replace if over 2 years old",
        "performance": "Update drivers, check for malware, increase RAM if needed",
        "turns off": "Check power supply connections, test battery health, verify thermal shutdown isn't occurring, update BIOS/firmware",
        "shutdown": "Verify power settings, check for overheating, test RAM modules, scan for malware",
        "unexpected": "Check event logs, verify hardware connections, test power supply, update system drivers",
        "power": "Test power adapter, check battery health, verify power settings, inspect charging port",
        "restart": "Check for Windows updates, scan for malware, test RAM, verify system temperature"
    }
    
    for key, solution in solutions.items():
        if key in issue.lower():
            return f"Technical Solution: {solution}"
    
    return "Please provide more details about the technical issue for specific troubleshooting steps."


print("✅ Agent tools created successfully")
print("🎯 Ready to create multi-agent orchestration system")


### Step 2: Define Agent Types and Communication Protocol

Let's establish the foundation for our multi-agent system by defining agent types and how they communicate.

In [ ]:
class AgentType(Enum):
    """Enumeration of available agent types"""
    ORCHESTRATOR = "orchestrator"
    CUSTOMER_SUPPORT = "customer_support"
    KNOWLEDGE_BASE = "knowledge_base"

class QueryType(Enum):
    """Classification of customer query types"""
    PRODUCT_INFO = "product_info"
    RETURN_POLICY = "return_policy"
    TECHNICAL_SUPPORT = "technical_support"
    GENERAL_SEARCH = "general_search"
    COMPLEX_MULTI_STEP = "complex_multi_step"

# Global agent registry for communication
AGENT_REGISTRY = {}

def register_agent(agent_type: AgentType, agent_instance: Agent):
    """Register an agent in the global registry"""
    AGENT_REGISTRY[agent_type.value] = agent_instance
    print(f"📝 Registered {agent_type.value} agent")

def get_agent(agent_type: AgentType) -> Agent:
    """Retrieve an agent from the registry"""
    return AGENT_REGISTRY.get(agent_type.value)

print("✅ Agent communication protocol established")

### Step 3: Create Orchestrator Tools

The orchestrator needs tools to analyze queries and route them to appropriate agents.

In [ ]:
@tool(
    name="analyze_customer_query",
    description="Analyze customer query to determine the best agent to handle it"
)
def analyze_customer_query(query: str) -> str:
    """Analyze customer query and return routing recommendation"""
    query_lower = query.lower()
    
    # Define routing logic based on keywords and patterns
    routing_rules = {
        QueryType.PRODUCT_INFO: [
            "product", "specification", "specs", "features", "price", "model", 
            "laptop", "phone", "headphone", "gaming", "console"
        ],
        QueryType.RETURN_POLICY: [
            "return", "refund", "exchange", "warranty", "policy", "money back"
        ],
        QueryType.TECHNICAL_SUPPORT: [
            "technical", "troubleshoot", "problem", "issue", "error", "bug", 
            "not working", "broken", "fix", "repair", "support", "help"
        ],
        QueryType.GENERAL_SEARCH: [
            "search", "find", "look up", "information", "latest", "news", "update"
        ]
    }
    
    # Score each query type based on keyword matches
    scores = {}
    for query_type, keywords in routing_rules.items():
        score = sum(1 for keyword in keywords if keyword in query_lower)
        if score > 0:
            scores[query_type] = score
    
    # Determine routing decision
    if not scores:
        recommended_agent = AgentType.CUSTOMER_SUPPORT.value
        confidence = "low"
        query_type = "general"
    else:
        top_query_type = max(scores.keys(), key=lambda x: scores[x])
        confidence = "high" if scores[top_query_type] >= 2 else "medium"
        query_type = top_query_type.value
        
        # Map query types to agents
        if top_query_type == QueryType.TECHNICAL_SUPPORT:
            recommended_agent = AgentType.KNOWLEDGE_BASE.value
        else:
            recommended_agent = AgentType.CUSTOMER_SUPPORT.value
    
    analysis = {
        "query": query,
        "query_type": query_type,
        "recommended_agent": recommended_agent,
        "confidence": confidence,
        "reasoning": f"Query classified as {query_type} with {confidence} confidence"
    }
    
    return json.dumps(analysis, indent=2)

@tool(
    name="route_to_customer_support",
    description="Route query to the Customer Support Agent for product info, returns, and general support"
)
def route_to_customer_support(query: str, context: str = "") -> str:
    """Route query to Customer Support Agent"""
    support_agent = get_agent(AgentType.CUSTOMER_SUPPORT)
    if not support_agent:
        return "❌ Customer Support Agent not available"
    
    # Add routing context to the query
    enhanced_query = f"{query}"
    if context:
        enhanced_query = f"Context: {context}\n\nCustomer Query: {query}"
    
    try:
        response = support_agent(enhanced_query)
        return f"🛠️ **Customer Support Agent Response:**\n{response.message['content'][0]['text']}"
    except Exception as e:
        return f"❌ Error from Customer Support Agent: {str(e)}"

@tool(
    name="route_to_knowledge_base",
    description="Route technical queries to the Knowledge Base Agent for deep technical support"
)
def route_to_knowledge_base(query: str, context: str = "") -> str:
    """Route query to Knowledge Base Agent"""
    kb_agent = get_agent(AgentType.KNOWLEDGE_BASE)
    if not kb_agent:
        return "❌ Knowledge Base Agent not available"
    
    # Add routing context to the query
    enhanced_query = f"{query}"
    if context:
        enhanced_query = f"Context: {context}\n\nTechnical Query: {query}"
    
    try:
        response = kb_agent(enhanced_query)
        return f"📚 **Knowledge Base Agent Response:**\n{response.message['content'][0]['text']}"
    except Exception as e:
        return f"❌ Error from Knowledge Base Agent: {str(e)}"

@tool(
    name="coordinate_multi_agent_response",
    description="Coordinate multiple agents for complex queries requiring multiple specializations"
)
def coordinate_multi_agent_response(query: str, agents_needed: str) -> str:
    """Coordinate multiple agents for complex queries"""
    agents_list = [agent.strip() for agent in agents_needed.split(",")]
    responses = []
    
    for agent_name in agents_list:
        if agent_name == "customer_support":
            response = route_to_customer_support(query, "Part of multi-agent coordination")
            responses.append(response)
        elif agent_name == "knowledge_base":
            response = route_to_knowledge_base(query, "Part of multi-agent coordination")
            responses.append(response)
    
    # Combine responses
    combined_response = "\n\n" + "="*50 + "\n\n".join(responses)
    return f"🤝 **Multi-Agent Coordinated Response:**\n{combined_response}"

print("✅ Orchestrator tools created successfully")

### Step 4: Create the Orchestrator Agent

The orchestrator is the entry point for all customer interactions. It analyzes queries and routes them to the most appropriate specialist agent.

In [ ]:
# Initialize the Bedrock model for orchestrator
orchestrator_model = BedrockModel(
    model_id=MODEL_ID,
    temperature=0.1,  # Lower temperature for more consistent routing decisions
    region_name=REGION
)

# Orchestrator system prompt
ORCHESTRATOR_SYSTEM_PROMPT = """
You are an intelligent Customer Support Orchestrator for an electronics e-commerce company. 
Your role is to analyze customer queries and route them to the most appropriate specialized agent.

**Available Specialist Agents:**

1. **Customer Support Agent** - Handles:
   - Product information and specifications
   - Return policies and warranty questions
   - General web searches for current information
   - Basic troubleshooting and support

2. **Knowledge Base Agent** - Handles:
   - Deep technical support and troubleshooting
   - Complex technical documentation queries
   - Specialized technical knowledge base searches
   - Advanced problem diagnosis

**Your Process:**
1. **Analyze** the customer query using `analyze_customer_query`
2. **Route** to the appropriate agent:
   - Use `route_to_customer_support` for product info, returns, general support
   - Use `route_to_knowledge_base` for technical issues and deep troubleshooting
   - Use `coordinate_multi_agent_response` for complex queries needing multiple agents
3. **Present** the response clearly to the customer

**Guidelines:**
- Always analyze the query first to make informed routing decisions
- For technical problems, prefer the Knowledge Base Agent
- For product questions and policies, prefer the Customer Support Agent
- For complex queries, coordinate multiple agents
- Be friendly and explain which specialist is helping them
- If routing fails, provide a helpful fallback response
"""

# Create the orchestrator agent
orchestrator_agent = Agent(
    model=orchestrator_model,
    tools=[
        analyze_customer_query,
        route_to_customer_support,
        route_to_knowledge_base,
        coordinate_multi_agent_response
    ],
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT
)

# Register the orchestrator
register_agent(AgentType.ORCHESTRATOR, orchestrator_agent)

print("✅ Orchestrator Agent created and registered successfully!")

### Step 5: Create the Customer Support Agent

This agent specializes in product information, return policies, and general customer support tasks.

In [ ]:
# Initialize the Bedrock model for customer support
support_model = BedrockModel(
    model_id=MODEL_ID,
    temperature=0.3,  # Balanced temperature for helpful responses
    region_name=REGION
)

# Customer Support Agent system prompt
CUSTOMER_SUPPORT_SYSTEM_PROMPT = """
You are a specialized Customer Support Agent for an electronics e-commerce company.
You are part of a multi-agent system and handle specific types of customer inquiries.

**Your Specializations:**
- Product information and specifications
- Return policies and warranty information
- General web searches for current information
- Basic troubleshooting and support guidance

**Available Tools:**
1. `get_product_info()` - Get detailed product specifications and information
2. `get_return_policy()` - Get return policy information for specific products
3. `web_search()` - Search the web for current information and updates

**Guidelines:**
- Use your tools to provide accurate, up-to-date information
- Be friendly, professional, and helpful
- For complex technical issues, acknowledge that the Knowledge Base Agent might be better suited
- Always offer additional assistance after answering questions
- Focus on your areas of expertise: products, policies, and general support
"""

# Create the customer support agent
customer_support_agent = Agent(
    model=support_model,
    tools=[
        get_product_info,
        get_return_policy,
        web_search
    ],
    system_prompt=CUSTOMER_SUPPORT_SYSTEM_PROMPT
)

# Register the customer support agent
register_agent(AgentType.CUSTOMER_SUPPORT, customer_support_agent)

print("✅ Customer Support Agent created and registered successfully!")

### Step 6: Create the Knowledge Base Agent

This agent specializes in technical support using the Bedrock Knowledge Base for deep technical knowledge.

In [ ]:
# Initialize the Bedrock model for knowledge base agent
kb_model = BedrockModel(
    model_id=MODEL_ID,
    temperature=0.2,  # Lower temperature for more precise technical responses
    region_name=REGION
)

# Knowledge Base Agent system prompt
KNOWLEDGE_BASE_SYSTEM_PROMPT = """
You are a specialized Knowledge Base Agent for technical support at an electronics e-commerce company.
You are part of a multi-agent system and focus exclusively on technical support and troubleshooting.

**Your Specialization:**
- Deep technical support and troubleshooting
- Complex technical problem diagnosis
- Advanced technical documentation and knowledge base queries
- Detailed technical guidance and solutions

**Available Tools:**
1. `get_technical_support()` - Access comprehensive technical knowledge base via Bedrock Knowledge Base

**Guidelines:**
- Use the technical knowledge base to provide accurate, detailed technical support
- Focus on technical problems, troubleshooting, and complex issues
- Provide step-by-step technical guidance when appropriate
- Be precise and technical in your responses while remaining helpful
- If a query is not technical in nature, acknowledge that other agents might be better suited
- Always verify technical information using your knowledge base tool
"""

# Create the knowledge base agent
knowledge_base_agent = Agent(
    model=kb_model,
    tools=[
        get_technical_support
    ],
    system_prompt=KNOWLEDGE_BASE_SYSTEM_PROMPT
)

# Register the knowledge base agent
register_agent(AgentType.KNOWLEDGE_BASE, knowledge_base_agent)

print("✅ Knowledge Base Agent created and registered successfully!")

### Step 7: Multi-Agent System Status

Let's verify that all agents are properly registered and ready to work together.

In [ ]:
def display_agent_status():
    """Display the status of all registered agents"""
    print("🤖 Multi-Agent System Status")
    print("=" * 50)
    
    # Hardcoded tool counts since Strands doesn't expose them easily
    expected_tools = {
        AgentType.ORCHESTRATOR: 0,        # Routes only, no tools
        AgentType.CUSTOMER_SUPPORT: 3,   # get_product_info, get_return_policy, web_search
        AgentType.KNOWLEDGE_BASE: 1      # get_technical_support
    }
    
    for agent_type in AgentType:
        agent = get_agent(agent_type)
        status = "✅ Online" if agent else "❌ Offline"
        
        if agent:
            tool_count = expected_tools.get(agent_type, 0)
            print(f"{agent_type.value.replace('_', ' ').title()}: {status} ({tool_count} tools)")
        else:
            print(f"{agent_type.value.replace('_', ' ').title()}: {status}")
    
    print("\n🔄 Agent Capabilities:")
    print("• Orchestrator: Query analysis, routing, coordination")
    print("• Customer Support: Product info, returns, web search")
    print("• Knowledge Base: Technical support, troubleshooting")
    print("\n✅ Multi-agent system ready for customer interactions!")
display_agent_status()


### Step 8: Test the Multi-Agent System

Now let's test our multi-agent system with different types of customer queries to see how the orchestrator routes them to appropriate specialists.

In [ ]:
def test_multi_agent_system(query: str, test_name: str):
    """Test the multi-agent system with a customer query"""
    print(f"\n🧪 **Test: {test_name}**")
    print(f"📝 Customer Query: \"{query}\"")
    print("-" * 60)
    
    try:
        # Route through orchestrator
        orchestrator = get_agent(AgentType.ORCHESTRATOR)
        response = orchestrator(query)
        print(response.message['content'][0]['text'])
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print("\n" + "=" * 80)

# Test 1: Product Information Query
test_multi_agent_system(
    "What are the specifications of your gaming laptops?",
    "Product Information Query"
)

In [ ]:
# Test 2: Technical Support Query
test_multi_agent_system(
    "Device turns off unexpectedly",
    "Technical Support Query"
)

In [ ]:
# Test 3: Return Policy Query
test_multi_agent_system(
    "What's the return policy for headphones if I don't like them?",
    "Return Policy Query"
)

In [ ]:
# Test 4: Complex Multi-Step Query
test_multi_agent_system(
    "I need a laptop for gaming under $1500, but I'm also having issues with my current laptop overheating. Can you help with both?",
    "Complex Multi-Step Query"
)

### Step 9: Interactive Multi-Agent Chat

Let's create an interactive session where you can chat with the multi-agent system.

## 🎉 Lab 1 Multi-Agent System Complete!

Congratulations! You've successfully built a **multi-agent customer support system** with intelligent orchestration. Here's what you accomplished:

### ✅ **What You Built**

1. **🎯 Orchestrator Agent**
   - Intelligent query analysis and routing
   - Multi-agent coordination capabilities
   - Fallback handling for edge cases

2. **🛠️ Customer Support Agent**
   - Product information specialist
   - Return policy expert
   - General web search capabilities

3. **📚 Knowledge Base Agent**
   - Technical support specialist
   - Deep troubleshooting expertise
   - Bedrock Knowledge Base integration

### 🏗️ **Key Architecture Benefits**

- **Specialization**: Each agent focuses on specific domains
- **Scalability**: Easy to add new specialist agents
- **Maintainability**: Clear separation of concerns
- **Intelligence**: Smart routing based on query analysis
- **Coordination**: Multi-agent responses for complex queries

### 🔄 **Agent Interaction Flow**

```
Customer Query → Orchestrator → Analysis → Route to Specialist → Response
                     ↓
              Multi-Agent Coordination (if needed)
```

### 🚀 **Next Steps**

Ready to enhance your multi-agent system? Continue with:

- **Lab 2**: Add persistent memory for multi-agent context sharing
- **Lab 3**: Implement AgentCore Gateway for centralized tool management
- **Lab 4**: Deploy to AgentCore Runtime for production scaling
- **Lab 5**: Build a multi-agent frontend interface

### 💡 **Key Learnings**

- **Agent Orchestration**: How to coordinate multiple specialized agents
- **Query Routing**: Intelligent analysis and routing of customer requests
- **Agent Specialization**: Benefits of domain-specific agents vs. monolithic agents
- **Multi-Agent Communication**: Patterns for agent-to-agent interaction

---

**🎊 Excellent work! You've built a sophisticated multi-agent system that demonstrates real-world enterprise patterns for AI agent orchestration!**

### 📖 **Continue Your Journey**

**[→ Proceed to Lab 2: Multi-Agent Memory](lab-02-multi-agent-memory.ipynb)**